In [1]:
# Import necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from api_credentials import client_id, client_secret
from functions import get_playlist_tracks_and_artists, find_tracks_positions_in_playlists
import json

# Load playlists from JSON file
with open('playlists.json', 'r') as file:
    playlists_dict = json.load(file)

# Initialise the Spotify client with client credentials for public data access
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

import logging
# Configure logging
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s')


## Fetch NMF tracks, and then find which playlists they are in 

In [2]:
# New Music Friday AU & NZ playlist 
playlist_id = '37i9dQZF1DWT2SPAYawYcO'

track_details = get_playlist_tracks_and_artists(sp, playlist_id)

track_positions = find_tracks_positions_in_playlists(sp, track_details, playlists_dict)

Checking playlist: Top 50 Australia
Checking playlist: New Music Friday AU & NZ
Checking playlist: Hot Hits Australia
Checking playlist: Front Left
Checking playlist: A1
Checking playlist: Dance Generation
Checking playlist: Get Popped!
Checking playlist: R&B Connect
Checking playlist: triple j's New Music Hitlist
Checking playlist: The Flavour
Checking playlist: Fresh Country
Checking playlist: New Dance Beats
Checking playlist: Pop n' Fresh
Checking playlist: R&B Feels
Checking playlist: Beats n' Bars
Checking playlist: Indie Arrivals
Checking playlist: Rock Out.
Checking playlist: Mellow Styles
Checking playlist: POP EDGE
Checking playlist: The Drip
Checking playlist: come thru
Checking playlist: Alt Here
Checking playlist: Breaking Hits
Checking playlist: Chilled Hits
Checking playlist: Vibe Check
Checking playlist: crush
Checking playlist: the hybrid
Checking playlist: Just Chill
Checking playlist: Morning Motivation
Checking playlist: Lazy Sunday
Checking playlist: Today's Top Hi

## Fetch Follower Count For Each Playlist

In [3]:
# Dictionary to store follower counts
playlist_followers = {}

# Fetching follower counts
for playlist_name, playlist_id in playlists_dict.items():
    playlist = sp.playlist(playlist_id)
    follower_count = playlist['followers']['total']
    playlist_followers[playlist_name] = follower_count

# # Printing the result

# for playlist_name, followers in playlist_followers.items():
#     print(f"{playlist_name}: {followers:,} followers")

## Create and save Dataframe 

In [4]:

rows = []

for track_id, track_info in track_positions.items():
    artist_name = track_info['artist_name']
    track_name = track_info['track_name']
    for playlist_info in track_info['playlists']:
        playlist_name = playlist_info['playlist']
        position = playlist_info['position']
        # Fetch the actual follower count using the playlist name
        followers = playlist_followers.get(playlist_name, 0)  # Default to 0 if playlist not found
        rows.append({
            'Artist': artist_name,
            'Title': track_name,
            'Playlist': playlist_name,
            'Position': position,
            'Followers': followers
        })

# Convert the list of rows into a DataFrame
df = pd.DataFrame(rows)

df.to_csv('streamlit.csv', index=False)

In [5]:
df.head()

,Artist,Title,Playlist,Position,Followers
0,Beyoncé,TEXAS HOLD 'EM,Top 50 Australia,4,815638
1,Beyoncé,TEXAS HOLD 'EM,New Music Friday AU & NZ,1,370460
2,Beyoncé,TEXAS HOLD 'EM,Hot Hits Australia,10,1400697
3,Beyoncé,TEXAS HOLD 'EM,triple j's New Music Hitlist,2,310290
4,Beyoncé,TEXAS HOLD 'EM,Fresh Country,1,378963
...,...,...,...,...,...
95,"Joel Corry, Majestic, Ron Carroll",Nikes On,Dance Generation,6,411017
96,"Joel Corry, Majestic, Ron Carroll",Nikes On,New Dance Beats,4,139680
97,ScHoolboy Q,Yeern 101,New Music Friday AU & NZ,28,370460
98,ScHoolboy Q,Yeern 101,Beats n' Bars,1,95088


### Troubleshooting incorrect positions being returned. 

In [ ]:
# track_id = "2wVV5aH5WMrQ797czLe0lU"
# playlist_id = "7vFQNWXoblEJXpbnTuyz76"

# def find_track_position(sp, playlist_id, track_id):
#     offset = 0
#     while True:
#         # Retrieve a page of tracks in the playlist
#         results = sp.playlist_items(playlist_id,
#                                     offset=offset,
#                                     fields='items.track.id,total')
#         # Check if we've reached the end of the playlist
#         if not results['items']:
#             break
#         # Iterate through the current page of tracks
#         for i, item in enumerate(results['items']):
#             current_track_id = item['track']['id']
#             if current_track_id == track_id:
#                 # Return the position corrected for 1-indexing
#                 return offset + i + 1  # Correcting for 1-indexing
#         # Move to the next page
#         offset += len(results['items'])
#     # If the track is not found after searching the entire playlist
#     return None

# # Find the track position
# position = find_track_position(sp, playlist_id, track_id)

# if position is not None:
#     print(f"Track found at position: {position}")
# else:
#     print("Track not found in the playlist.")

